<a href="https://colab.research.google.com/github/shz1236/Remote-sensing-pattern-recognition/blob/master/LE_NET5_tang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
pip install tensorflow-gpu

     |████████████████████████████████| 421.8MB 30kB/s 
     |████████████████████████████████| 3.8MB 41.8MB/s 
     |████████████████████████████████| 450kB 59.6MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found exis

In [2]:
import tensorflow as tf
import numpy as np
import os
small_batch= True

num_classes=10

learning_rate = 0.01
training_steps = 1000
batch_size=500

conv1_filters = 6
conv2_filters = 16
conv3_filters = 120
fc1_units = 84

optimizer = tf.optimizers.SGD(learning_rate)
os.chdir("/content/drive/My Drive/Colab Notebooks/")

from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train / 255., x_test / 255.



train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().batch(batch_size).prefetch(1)

def conv2d(x, W, b, strides=1,padding='SAME'):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding=padding)
    x = tf.nn.bias_add(x, b)
    return x

def maxpool2d(x, k=2,padding='SAME'):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

weights,biases=np.load('WB.npy',allow_pickle=True)

# random_normal = tf.initializers.RandomNormal()
# weights = {
#     'wc1': tf.Variable(random_normal([5, 5, 1, conv1_filters])),
#     'wc2': tf.Variable(random_normal([5, 5, conv1_filters, conv2_filters])),
#     'wc3': tf.Variable(random_normal([5, 5, conv2_filters, conv3_filters])),

#     'wd1': tf.Variable(random_normal([conv3_filters, fc1_units])),
#     'out': tf.Variable(random_normal([fc1_units, num_classes]))
# }

# biases = {
#     'bc1': tf.Variable(tf.zeros([conv1_filters])),
#     'bc2': tf.Variable(tf.zeros([conv2_filters])),
#     'bc3': tf.Variable(tf.zeros([conv3_filters])),

#     'bd1': tf.Variable(tf.zeros([fc1_units])),
#     'out': tf.Variable(tf.zeros([num_classes]))
# }

# np.save('WB.npy',(weights,biases),allow_pickle=True)

    
def bincode(input,output):
    input=tf.where(input>0,1.0,0.0)
    output_bincode=tf.where(output>0,1.0,0.0)
    output_bincode=tf.expand_dims(output_bincode,3)
    output_bincode=tf.tile(output_bincode, multiples=[1,1,1,input.shape[-1],1])
    layer_bincode=np.array([[x,y] for x,y in zip(input,output_bincode)])
    layer_bincode=np.apply_along_axis(lambda x : tf.nn.conv2d(np.expand_dims(x[0],0) ,x[1],strides=[1, 1, 1, 1], padding='VALID'),1,layer_bincode)
    layer_bincode=tf.where(layer_bincode==0.0,5.0,layer_bincode)
    layer_bincode=tf.tile(tf.transpose(layer_bincode,perm=[0,2,3,1,4]),multiples=[1,1,1,input.shape[-1],1])
    return layer_bincode

def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return y_true * tf.math.log(y_pred)

    # return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

def run_optimization(x, y):
    trainable_variables = list(weights.values()) + list(biases.values())
    with tf.GradientTape(persistent=True) as g:
            x = tf.reshape(x, [-1, 28, 28, 1])
            x = tf.pad(x,[[0,0],[2,2],[2,2],[0,0]])

            conv1 = conv2d(x, weights['wc1'], biases['bc1'],padding='VALID')
            act1 = tf.nn.relu(conv1)
            pool1 = maxpool2d(act1, k=2)

            conv2 = conv2d(pool1, weights['wc2'], biases['bc2'],padding='VALID')
            act2 = tf.nn.relu(conv2) 
            pool2 = maxpool2d(act2, k=2)

            conv3 = conv2d(pool2, weights['wc3'], biases['bc3'],padding='VALID')
            act3 = tf.nn.relu(conv3)

            fc1 = tf.reshape(act3, [-1, weights['wd1'].get_shape().as_list()[0]])
            fc1_bincode=tf.reduce_sum(tf.where(fc1>0 ,1.0,0.0),axis=0)

            fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
            fc1 = tf.nn.relu(fc1)
            fc1_bincode=tf.matmul(tf.expand_dims(fc1_bincode,axis=1), tf.expand_dims(tf.ones_like(biases['bd1']),axis=0))
            fc1_bincode=tf.where(fc1_bincode==0,5,fc1_bincode)
            out_bincode=tf.reduce_sum(tf.where(fc1>0 ,1.0,0.0),axis=0)

            out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
            out = tf.nn.softmax(out)
            out_bincode=tf.matmul(tf.expand_dims(out_bincode,axis=1), tf.expand_dims(tf.ones_like(biases['out']),axis=0))
            out_bincode=tf.where(out_bincode==0,5,out_bincode)
           
            loss = -tf.reduce_sum(cross_entropy(out, y),axis=1)
            gradients=list(map(lambda x:list(map(lambda x:x.numpy(),g.gradient(x, trainable_variables))),loss))
    conv1_delta = g.gradient(loss, conv1)
    conv2_delta = g.gradient(loss, conv2)
    conv3_delta = g.gradient(loss, conv3)
    wc1_bincode=bincode(x,conv1_delta)
    wc2_bincode=bincode(pool1,conv2_delta)
    wc3_bincode=bincode(pool2,conv3_delta)
    layers_bincode=(wc1_bincode,wc2_bincode,wc3_bincode,fc1_bincode,out_bincode)
    gradient = g.gradient(loss, trainable_variables)

    if small_batch==True:
        wc1_bincode=tf.reduce_sum(np.array([x[0] for x in gradients])/layers_bincode[0],0)
        wc2_bincode=tf.reduce_sum(np.array([x[1] for x in gradients])/layers_bincode[1],0)
        wc3_bincode=tf.reduce_sum(np.array([x[2] for x in gradients])/layers_bincode[2],0)
        gradient[0:3]=[wc1_bincode,wc2_bincode,wc3_bincode]
        gradient[3:5]=[tf.divide(x,y) for x,y in zip(gradient[3:5],layers_bincode[3:5])]
    else:
        gradient[0:5]=[tf.divide(x,batch_size) for x in gradient[0:5]]
              
    optimizer.apply_gradients(zip(gradient, trainable_variables))
    return out

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
loss_list=[]
acc_list=[]
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    pred=run_optimization(batch_x, batch_y)
    loss = tf.reduce_mean(-tf.reduce_sum(cross_entropy(pred, batch_y)))
    acc = accuracy(pred, batch_y)
    print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))
    loss_list.append(loss)
    acc_list.append(acc)

if  small_batch==True:

    np.save('training_accuracy_small_batchsize_epoch=%d_batchsize=%d_learn_rate=%f.npy' % (
                training_steps, batch_size, learning_rate),
            acc_list, allow_pickle=True)
    np.save('training_cost_small_batchsize_epoch=%d_batchsize=%d_learn_rate=%f.npy' % (
                training_steps, batch_size, learning_rate),
            loss_list, allow_pickle=True)
else:
    np.save('training_accuracy_normal_batchsize_epoch=%d_batchsize=%d_learn_rate=%f.npy' % (
                training_steps, batch_size, learning_rate),
            acc_list, allow_pickle=True)
    np.save('training_cost_normal_batchsize_epoch=%d_batchsize=%d_learn_rate=%f.npy' % (
                training_steps, batch_size, learning_rate),
            loss_list, allow_pickle=True)

step: 1, loss: 1152.089600, accuracy: 0.072000
step: 2, loss: 1152.331055, accuracy: 0.100000
step: 3, loss: 1148.860352, accuracy: 0.120000
step: 4, loss: 1151.642090, accuracy: 0.094000
step: 5, loss: 1150.067627, accuracy: 0.138000
step: 6, loss: 1149.866821, accuracy: 0.090000
step: 7, loss: 1151.921631, accuracy: 0.104000
step: 8, loss: 1149.240479, accuracy: 0.088000
step: 9, loss: 1148.140625, accuracy: 0.180000
step: 10, loss: 1145.484619, accuracy: 0.112000
step: 11, loss: 1147.256836, accuracy: 0.102000
step: 12, loss: 1146.739258, accuracy: 0.102000
